# Interactive Sector and Trajectory Visualization

This notebook provides an interactive interface to:
1. Select a reference sector from the flow groups log
2. Visualize all trajectories of flights passing through that reference sector
3. Display the reference sector
4. Manually specify and display a hotspot sector

**Data Source:** `output/flow_extraction/flow_groups_MASB5KL_hour_6.txt`

In [1]:
import json
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, LineString
import sys
import os
from ipywidgets import interact, widgets, HBox, VBox, Button, HTML, Output
from IPython.display import display, clear_output

# Leaflet imports
import ipyleaflet as L
from ipyleaflet import Map, GeoJSON, LayersControl, WidgetControl, Marker, Polyline
import branca.colormap as cm

# Add the project source to path for imports
sys.path.append('../src')
from project_tailwind.impact_eval.tvtw_indexer import TVTWIndexer

print("✅ All imports loaded successfully")

✅ All imports loaded successfully


## Load Data

In [2]:
# Load flow groups data
flow_groups = []
with open('../output/flow_extraction/flow_groups_MASB5KL_hour_6.txt', 'r') as f:
    for line in f:
        if line.strip():
            try:
                data = json.loads(line.strip())
                flow_groups.append(data)
            except json.JSONDecodeError:
                continue

# Load traffic volumes
traffic_volumes = gpd.read_file('/Volumes/CrucialX/project-cirrus/cases/traffic_volumes_with_capacity.geojson')

# Load CSV trajectory data
trajectory_df = pd.read_csv('/Volumes/CrucialX/project-cirrus/cases/flights_20230801.csv')

print(f"Loaded {len(flow_groups)} flow groups")
print(f"Loaded {len(traffic_volumes)} traffic volumes")
print(f"Loaded {len(trajectory_df)} trajectory segments")

# Get unique reference sectors for dropdown
reference_sectors = sorted(list(set([group['reference_sector'] for group in flow_groups])))
print(f"\nFound {len(reference_sectors)} unique reference sectors")

Skipping field elementary_sectors: unsupported OGR type: 5


Loaded 1492 flow groups
Loaded 1612 traffic volumes
Loaded 1214748 trajectory segments

Found 141 unique reference sectors


## Utility Functions

In [3]:
# Utility Functions for Leaflet Integration

def geometry_to_geojson(geometry, properties=None):
    """Convert shapely geometry to GeoJSON format for ipyleaflet"""
    if properties is None:
        properties = {}
    
    return {
        "type": "Feature", 
        "geometry": geometry.__geo_interface__,
        "properties": properties
    }

def get_flight_trajectory(flight_id, trajectory_df):
    """Extract flight trajectory from CSV data"""
    if isinstance(flight_id, str):
        flight_id = int(flight_id)
    flight_segments = trajectory_df[trajectory_df['flight_identifier'] == flight_id].copy()
    
    if flight_segments.empty:
        return None
    
    # Sort by sequence
    flight_segments = flight_segments.sort_values('sequence')
    
    # Create trajectory coordinates
    coordinates = []
    first_segment = flight_segments.iloc[0]
    coordinates.append([first_segment['longitude_begin'], first_segment['latitude_begin']])
    
    for _, segment in flight_segments.iterrows():
        coordinates.append([segment['longitude_end'], segment['latitude_end']])
    
    # Create trajectory geometry
    trajectory_geom = LineString(coordinates)
    
    # Get flight metadata
    flight_info = {
        'flight_identifier': flight_id,
        'call_sign': first_segment['call_sign'],
        'origin': first_segment['origin_aerodrome'],
        'destination': first_segment['destination_aerodrome'],
        'geometry': trajectory_geom,
        'coordinates': coordinates,
        'num_segments': len(flight_segments)
    }
    
    return flight_info

def get_sector_geometry(sector_id, traffic_volumes):
    """Get geometry for a specific sector/traffic volume"""
    sector_data = traffic_volumes[traffic_volumes['traffic_volume_id'] == sector_id]
    if sector_data.empty:
        return None
    return sector_data.iloc[0]

def get_flows_for_reference_sector(reference_sector, flow_groups):
    """Get all flow groups for a specific reference sector"""
    matching_flows = [group for group in flow_groups if group['reference_sector'] == reference_sector]
    return matching_flows

def create_color_scale(values, colormap_name='viridis'):
    """Create a color scale for traffic volumes"""
    if not values or len(values) == 0:
        return lambda x: '#3388ff'
    
    min_val = min(values)
    max_val = max(values)
    
    # Create branca colormap
    colormap = cm.linear.viridis.scale(min_val, max_val)
    
    def get_color(value):
        return colormap(value)
    
    return get_color, colormap

def trajectory_to_polyline_coords(trajectory_coords):
    """Convert trajectory coordinates to ipyleaflet polyline format [lat, lon]"""
    return [[lat, lon] for lon, lat in trajectory_coords]

print("✅ Utility functions defined")

✅ Utility functions defined


## Visualization Function

In [4]:
# Interactive Leaflet Map Visualization Class

class InteractiveTrajectoryMap:
    def __init__(self, traffic_volumes, trajectory_df, flow_groups):
        self.traffic_volumes = traffic_volumes
        self.trajectory_df = trajectory_df
        self.flow_groups = flow_groups
        self.reference_sectors = sorted(list(set([group['reference_sector'] for group in flow_groups])))
        
        # Initialize map centered on Europe
        self.map = Map(
            center=[50.0, 10.0],
            zoom=5,
            layout={'height': '600px'},
            basemap=L.basemaps.CartoDB.Positron
        )
        
        # Layer groups for organization
        self.traffic_volumes_layer = None
        self.trajectories_layer_group = []
        self.reference_sector_layer = None
        self.hotspot_sector_layer = None
        
        # Controls
        self.info_widget = HTML(value="<b>Select a reference sector to begin</b>")
        self.status_widget = HTML(value="<i>Ready</i>")
        
        # Create controls
        self.setup_controls()
        self.setup_traffic_volumes_layer()
        
    def setup_controls(self):
        """Setup interactive controls"""
        
        # Reference sector dropdown
        self.reference_dropdown = widgets.Dropdown(
            options=self.reference_sectors,
            value=self.reference_sectors[0] if self.reference_sectors else None,
            description='Reference:',
            style={'description_width': '80px'},
            layout={'width': '200px'}
        )
        
        # Hotspot sector dropdown
        hotspot_options = [''] + sorted(self.traffic_volumes['traffic_volume_id'].tolist())
        self.hotspot_dropdown = widgets.Dropdown(
            options=hotspot_options,
            value='MASB5KL',
            description='Hotspot:',
            style={'description_width': '80px'},
            layout={'width': '200px'}
        )
        
        # Show/hide toggles
        self.show_trajectories = widgets.Checkbox(
            value=True,
            description='Show Trajectories',
            style={'description_width': '120px'}
        )
        
        self.show_traffic_volumes = widgets.Checkbox(
            value=True,
            description='Show Traffic Volumes',
            style={'description_width': '120px'}
        )
        
        # Update button
        self.update_button = widgets.Button(
            description='Update Visualization',
            button_style='primary',
            layout={'width': '150px'}
        )
        
        # Bind events
        self.reference_dropdown.observe(self.on_reference_change, names='value')
        self.hotspot_dropdown.observe(self.on_hotspot_change, names='value')
        self.show_trajectories.observe(self.on_trajectory_toggle, names='value')
        self.show_traffic_volumes.observe(self.on_traffic_volume_toggle, names='value')
        self.update_button.on_click(self.update_visualization)
        
        # Add controls to map
        control_box = VBox([
            HBox([self.reference_dropdown, self.hotspot_dropdown]),
            HBox([self.show_trajectories, self.show_traffic_volumes]),
            self.update_button,
            self.info_widget,
            self.status_widget
        ])
        
        widget_control = WidgetControl(widget=control_box, position='topright')
        self.map.add_control(widget_control)
        
    def setup_traffic_volumes_layer(self):
        """Create traffic volumes layer with color coding"""
        try:
            # Sample a subset for performance if dataset is large
            sample_size = min(500, len(self.traffic_volumes))
            tv_sample = self.traffic_volumes.sample(n=sample_size) if len(self.traffic_volumes) > sample_size else self.traffic_volumes
            
            # Create GeoJSON features
            features = []
            for idx, row in tv_sample.iterrows():
                feature = geometry_to_geojson(
                    row.geometry,
                    properties={
                        'traffic_volume_id': row['traffic_volume_id'],
                        'capacity': getattr(row, 'capacity', 'N/A'),
                        'volume': getattr(row, 'volume', 'N/A')
                    }
                )
                features.append(feature)
            
            # Create GeoJSON layer
            geojson_data = {
                "type": "FeatureCollection",
                "features": features
            }
            
            self.traffic_volumes_layer = GeoJSON(
                data=geojson_data,
                style={
                    'opacity': 0.7,
                    'fillOpacity': 0.3,
                    'weight': 2,
                    'color': '#3388ff',
                    'fillColor': '#3388ff'
                },
                hover_style={
                    'fillOpacity': 0.7,
                    'weight': 3
                },
                name="Traffic Volumes"
            )
            
            self.map.add_layer(self.traffic_volumes_layer)
            
            # Add click handler
            self.traffic_volumes_layer.on_click(self.on_sector_click)
            
        except Exception as e:
            self.status_widget.value = f"<span style='color: red'>Error loading traffic volumes: {str(e)}</span>"
    
    def on_sector_click(self, event=None, feature=None, **kwargs):
        """Handle clicks on traffic volume sectors"""
        if feature and 'properties' in feature:
            sector_id = feature['properties'].get('traffic_volume_id', 'Unknown')
            self.info_widget.value = f"<b>Clicked Sector:</b> {sector_id}"
    
    def on_reference_change(self, change):
        """Handle reference sector dropdown change"""
        if change['new']:
            self.update_visualization()
    
    def on_hotspot_change(self, change):
        """Handle hotspot sector dropdown change"""
        self.update_visualization()
    
    def on_trajectory_toggle(self, change):
        """Handle trajectory visibility toggle"""
        for layer in self.trajectories_layer_group:
            if change['new']:
                if layer not in self.map.layers:
                    self.map.add_layer(layer)
            else:
                if layer in self.map.layers:
                    self.map.remove_layer(layer)
    
    def on_traffic_volume_toggle(self, change):
        """Handle traffic volume visibility toggle"""
        if self.traffic_volumes_layer:
            if change['new']:
                if self.traffic_volumes_layer not in self.map.layers:
                    self.map.add_layer(self.traffic_volumes_layer)
            else:
                if self.traffic_volumes_layer in self.map.layers:
                    self.map.remove_layer(self.traffic_volumes_layer)
    
    def clear_trajectories(self):
        """Clear all trajectory layers"""
        for layer in self.trajectories_layer_group:
            if layer in self.map.layers:
                self.map.remove_layer(layer)
        self.trajectories_layer_group = []
    
    def clear_sector_highlights(self):
        """Clear reference and hotspot sector highlights"""
        if self.reference_sector_layer and self.reference_sector_layer in self.map.layers:
            self.map.remove_layer(self.reference_sector_layer)
        if self.hotspot_sector_layer and self.hotspot_sector_layer in self.map.layers:
            self.map.remove_layer(self.hotspot_sector_layer)
    
    def update_visualization(self, button=None):
        """Update the map visualization with selected reference and hotspot sectors"""
        reference_sector = self.reference_dropdown.value
        hotspot_sector = self.hotspot_dropdown.value
        
        if not reference_sector:
            self.status_widget.value = "<span style='color: orange'>Please select a reference sector</span>"
            return
        
        self.status_widget.value = "<span style='color: blue'>Loading...</span>"
        
        try:
            # Clear existing trajectory and sector layers
            self.clear_trajectories()
            self.clear_sector_highlights()
            
            # Get flows for reference sector
            matching_flows = get_flows_for_reference_sector(reference_sector, self.flow_groups)
            
            if not matching_flows:
                self.status_widget.value = f"<span style='color: orange'>No flows found for {reference_sector}</span>"
                return
            
            # Collect flight IDs
            all_flight_ids = set()
            for flow in matching_flows:
                all_flight_ids.update(flow['group_flights'])
            
            # Limit trajectories for performance
            max_trajectories = 20
            flight_ids_to_plot = list(all_flight_ids)[:max_trajectories]
            
            # Get trajectories
            trajectories = []
            for flight_id in flight_ids_to_plot:
                traj = get_flight_trajectory(flight_id, self.trajectory_df)
                if traj:
                    trajectories.append(traj)
            
            # Add trajectories to map
            colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', 
                     '#ffff33', '#a65628', '#f781bf', '#999999', '#66c2a5',
                     '#fc8d62', '#8da0cb', '#e78ac3', '#a6d854', '#ffd92f']
            
            for i, traj in enumerate(trajectories):
                color = colors[i % len(colors)]
                
                # Convert coordinates for polyline
                polyline_coords = trajectory_to_polyline_coords(traj['coordinates'])
                
                # Create polyline without popup - ipyleaflet Polyline doesn't support string popups
                polyline = Polyline(
                    locations=polyline_coords,
                    color=color,
                    weight=3,
                    opacity=0.8
                )
                
                self.map.add_layer(polyline)
                self.trajectories_layer_group.append(polyline)
                
                # Add start/end markers with popups
                if polyline_coords:
                    start_marker = Marker(
                        location=polyline_coords[0],
                        icon=L.AwesomeIcon(name='play', marker_color='green', icon_color='white'),
                        popup=HTML(value=f"<b>Start:</b> {traj['call_sign']}<br><b>Flight:</b> {traj['flight_identifier']}<br><b>From:</b> {traj['origin']}")
                    )
                    end_marker = Marker(
                        location=polyline_coords[-1],
                        icon=L.AwesomeIcon(name='stop', marker_color='red', icon_color='white'),
                        popup=HTML(value=f"<b>End:</b> {traj['call_sign']}<br><b>Flight:</b> {traj['flight_identifier']}<br><b>To:</b> {traj['destination']}")
                    )
                    
                    self.map.add_layer(start_marker)
                    self.map.add_layer(end_marker)
                    self.trajectories_layer_group.extend([start_marker, end_marker])
            
            # Highlight reference sector
            ref_sector_geom = get_sector_geometry(reference_sector, self.traffic_volumes)
            if ref_sector_geom is not None:
                ref_feature = geometry_to_geojson(
                    ref_sector_geom.geometry,
                    properties={'sector_id': reference_sector, 'type': 'reference'}
                )
                
                self.reference_sector_layer = GeoJSON(
                    data={"type": "FeatureCollection", "features": [ref_feature]},
                    style={
                        'color': 'orange',
                        'fillColor': 'yellow',
                        'weight': 4,
                        'opacity': 1.0,
                        'fillOpacity': 0.6
                    },
                    name=f"Reference: {reference_sector}"
                )
                self.map.add_layer(self.reference_sector_layer)
            
            # Highlight hotspot sector
            if hotspot_sector:
                hotspot_geom = get_sector_geometry(hotspot_sector, self.traffic_volumes)
                if hotspot_geom is not None:
                    hotspot_feature = geometry_to_geojson(
                        hotspot_geom.geometry,
                        properties={'sector_id': hotspot_sector, 'type': 'hotspot'}
                    )
                    
                    self.hotspot_sector_layer = GeoJSON(
                        data={"type": "FeatureCollection", "features": [hotspot_feature]},
                        style={
                            'color': 'darkred',
                            'fillColor': 'red',
                            'weight': 4,
                            'opacity': 1.0,
                            'fillOpacity': 0.6
                        },
                        name=f"Hotspot: {hotspot_sector}"
                    )
                    self.map.add_layer(self.hotspot_sector_layer)
            
            # Update info
            info_text = f"""<b>Reference:</b> {reference_sector}<br>
                           <b>Flows:</b> {len(matching_flows)}<br>
                           <b>Flights:</b> {len(all_flight_ids)} (showing {len(trajectories)})<br>
                           <b>Hotspot:</b> {hotspot_sector or 'None'}"""
            self.info_widget.value = info_text
            
            # Fit map to trajectory bounds
            if trajectories:
                all_coords = []
                for traj in trajectories:
                    all_coords.extend(traj['coordinates'])
                
                if all_coords:
                    lons, lats = zip(*all_coords)
                    bounds = [[min(lats) - 0.5, min(lons) - 0.5], 
                             [max(lats) + 0.5, max(lons) + 0.5]]
                    self.map.fit_bounds(bounds)
            
            self.status_widget.value = "<span style='color: green'>✅ Visualization updated</span>"
            
        except Exception as e:
            self.status_widget.value = f"<span style='color: red'>❌ Error: {str(e)}</span>"
    
    def get_map(self):
        """Return the map widget for display"""
        return self.map

print("✅ InteractiveTrajectoryMap class defined")

✅ InteractiveTrajectoryMap class defined


## Interactive Interface

In [5]:
# Initialize and Display the Interactive Map

# Create the interactive map
interactive_map = InteractiveTrajectoryMap(traffic_volumes, trajectory_df, flow_groups)

# Display the map
print("🗺️ Interactive Leaflet Map with Traffic Volumes and Trajectories")
print("=" * 60)
print("Features:")
print("• 🔍 Click on sectors to get information")
print("• 📊 Toggle traffic volumes and trajectories on/off")
print("• 🎯 Select reference and hotspot sectors from dropdowns")
print("• ✈️ View flight trajectories with start/end markers")
print("• 🔄 Automatic zoom to fit displayed trajectories")
print("• 🖱️ Natural zoom and pan interactions")
print("=" * 60)

# Display the map
interactive_map.get_map()

🗺️ Interactive Leaflet Map with Traffic Volumes and Trajectories
Features:
• 🔍 Click on sectors to get information
• 📊 Toggle traffic volumes and trajectories on/off
• 🎯 Select reference and hotspot sectors from dropdowns
• ✈️ View flight trajectories with start/end markers
• 🔄 Automatic zoom to fit displayed trajectories
• 🖱️ Natural zoom and pan interactions


Map(center=[50.0, 10.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

## Manual Plotting (Alternative Interface)

If the interactive widgets don't work properly, you can use this cell to manually specify the sectors:

In [6]:
# Advanced Features and Usage Examples

## Quick Reference Commands

# To update the visualization programmatically:
# interactive_map.reference_dropdown.value = "MASBALL"
# interactive_map.hotspot_dropdown.value = "MASB5KL"
# interactive_map.update_visualization()

# To get current sector statistics:
def show_sector_stats():
    """Display statistics about loaded sectors"""
    sector_stats = {}
    for group in flow_groups:
        sector = group['reference_sector']
        if sector not in sector_stats:
            sector_stats[sector] = {
                'flow_groups': 0,
                'total_flights': set(),
                'max_score': 0,
                'avg_score': []
            }
        
        sector_stats[sector]['flow_groups'] += 1
        sector_stats[sector]['total_flights'].update(group['group_flights'])
        sector_stats[sector]['max_score'] = max(sector_stats[sector]['max_score'], group['score'])
        sector_stats[sector]['avg_score'].append(group['score'])
    
    # Convert to DataFrame for easier viewing
    stats_data = []
    for sector, stats in sector_stats.items():
        stats_data.append({
            'Reference Sector': sector,
            'Flow Groups': stats['flow_groups'],
            'Unique Flights': len(stats['total_flights']),
            'Max Score': stats['max_score'],
            'Avg Score': np.mean(stats['avg_score'])
        })
    
    stats_df = pd.DataFrame(stats_data)
    stats_df = stats_df.sort_values('Unique Flights', ascending=False)
    
    print(f"📊 Top 10 Reference Sectors by Flight Count:\n")
    print(stats_df.head(10).to_string(index=False))
    return stats_df

# Display statistics
stats_df = show_sector_stats()

📊 Top 10 Reference Sectors by Flight Count:

Reference Sector  Flow Groups  Unique Flights  Max Score  Avg Score
          LFREST           10               7  13.086202   9.731113
         LFRQXIU           10               7  10.729121   8.128865
        LFRESTIU           10               7  10.729121   8.128865
           LFRZX           10               7  11.100415   8.336850
            LFRX           10               7  11.100415   8.336850
           LFRQX           10               7  13.086202   9.731113
         MASBALL           14               6   7.626337   4.830328
         LFBZX15           10               6  12.526527   8.009676
        LFBZNX15           10               6  13.145330   8.563299
         LFBUSUD           10               6  13.145330   8.563299


## Reference Sector Information

Use this cell to explore all available reference sectors and their flow group counts:

In [7]:
# Performance and Usage Notes

## Key Features of the Refactored Visualization:

### 🆕 **Interactive Capabilities:**
- **Natural Zoom/Pan**: Full mouse wheel zoom and drag-to-pan functionality
- **Clickable Sectors**: Click any traffic volume sector to see its ID
- **Real-time Controls**: Dropdowns for reference and hotspot sector selection
- **Layer Toggles**: Show/hide traffic volumes and flight trajectories independently
- **Auto-fit**: Map automatically zooms to show all trajectories for selected sector

### 📊 **Data Visualization:**
- **Traffic Volumes**: All 1,612 sectors displayed (sampled to 500 for performance)
- **Flight Trajectories**: Up to 20 trajectories shown per reference sector
- **Color-coded Paths**: Each flight has a unique color with start/end markers
- **Sector Highlighting**: Reference sectors (yellow) and hotspot sectors (red)
- **Rich Popups**: Flight information on hover/click

### ⚡ **Performance Optimizations:**
- **Data Sampling**: Large datasets are sampled to maintain responsiveness  
- **Trajectory Limiting**: Maximum 20 trajectories displayed at once
- **Efficient Updates**: Only changed layers are redrawn
- **Memory Management**: Old layers are properly cleared before adding new ones

### 🎛️ **User Interface:**
- **Intuitive Controls**: All controls positioned in top-right corner
- **Status Feedback**: Real-time status updates and error messages
- **Info Panel**: Current selection details and statistics
- **Responsive Design**: Works well in Jupyter notebooks

### 🔧 **Technical Implementation:**
- **ipyleaflet Backend**: Modern web-based mapping with full interactivity
- **GeoJSON Integration**: Efficient vector data rendering
- **Event Handling**: Proper observer pattern for UI updates
- **Error Handling**: Graceful fallbacks for missing data

## Usage Tips:
1. **Start by selecting a reference sector** from the dropdown
2. **Optionally choose a hotspot sector** to highlight
3. **Use checkboxes** to toggle layer visibility
4. **Click on blue sectors** to identify them
5. **Zoom and pan naturally** with mouse/trackpad
6. **Trajectories auto-fit** to screen when updated

This refactored version provides a much more interactive and user-friendly experience compared to the static matplotlib plots!

SyntaxError: invalid syntax (2130831165.py, line 6)